In [3]:
import os
import whitebox
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.features import rasterize
import numpy as np
import pathlib 
from datetime import datetime


Utility Functions

In [1]:
from pathlib import Path

def setup_project_directories(base_dir=None): 
    """
    Creates a structured project directory with default folders. Ensures existing directories 
    with data are not overwritten. Returns a dictionary of created directories.

    Parameters:
    - base_dir (str or Path, optional): Path to the base directory. If None, asks user for input.

    Returns:
    - dict: Dictionary containing the paths of created directories.
    """

    # Assign or ask for base directory
    if base_dir is None:
        base_dir = input(f"Enter the base directory (default: {Path.cwd()}): ").strip()
        base_dir = Path(base_dir) if base_dir else Path.cwd()
    else:
        base_dir = Path(base_dir)

    # Validate path
    if not base_dir.exists():
        print(f" Warning: The specified base directory does not exist. Creating it at: {base_dir}")
        base_dir.mkdir(parents=True, exist_ok=True)

    # Define useful subdirectories
    subdirs = {
        "Data": base_dir / "data",
        "Raw Data": base_dir / "data" / "raw",
        "Processed Data": base_dir / "data" / "processed",
        "Output": base_dir / "output",
        "Figures": base_dir / "output" / "figures",
        "Logs": base_dir / "logs",
        "Config": base_dir / "config",
        "Scripts": base_dir / "scripts",
        "Notebooks": base_dir / "notebooks",
        "Reports": base_dir / "reports",
    }

    # Track created directories
    created_dirs = {}

    # Create directories safely
    for name, path in subdirs.items():
        if path.exists() and any(path.iterdir()):  # Check if directory exists and has files
            print(f"Warning: '{name}' directory already exists and contains data. Skipping creation.")
        else:
            path.mkdir(parents=True, exist_ok=True)
            created_dirs[name] = path  # Store successfully created directories

    # Print Summary
    print(f"Base Directory: {base_dir.resolve()}\n")

    for name, path in subdirs.items():
        status = "✅ Created" if name in created_dirs else "⚠️ Exists (Not Modified)"
        print(f"   📁 {name}: {path.relative_to(base_dir)}  - {status}")

    print("\n Setup complete!")

    '''
    # Example Usage
    project_dirs = setup_project_directories("/Users/yourname/projects")

    # Use returned dictionary for path references
    dem_path = project_dirs["Data"] / "input_dem.tif"
    print(f"\nDEM will be stored in: {dem_path}")
    '''
    return subdirs  # Return the full dictionary of directories




In [2]:
project_dirs = setup_project_directories(r'\Users\ah5766\OneDrive - Halff\Documents\Code\proj\wbt_lidar')

Base Directory: C:\Users\ah5766\OneDrive - Halff\Documents\Code\proj\wbt_lidar

   📁 Data: data  - ✅ Created
   📁 Raw Data: data\raw  - ✅ Created
   📁 Processed Data: data\processed  - ✅ Created
   📁 Output: output  - ✅ Created
   📁 Figures: output\figures  - ✅ Created
   📁 Logs: logs  - ✅ Created
   📁 Config: config  - ✅ Created
   📁 Scripts: scripts  - ✅ Created
   📁 Notebooks: notebooks  - ✅ Created
   📁 Reports: reports  - ✅ Created

 Setup complete!


In [6]:
manifest_path = project_dirs['Data'] / 'usgs_lpc_livingston_amite_2018.csv'

In [9]:
manifest = pd.read_csv(manifest_path, header = None)
manifest

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,USGS Original Project Resolution LA_Amite_2018...,This is the Original Product Resolution (OPR) ...,623b8a35d34e915b67d019a1,ScienceBase,NaN,gda,https://www.sciencebase.gov/catalog/item/623b8...,https://prd-tnm.s3.amazonaws.com/index.html?pr...,2019-11-07,2022-03-23T15:24:48.073-06:00,...,NaN,https://prd-tnm.s3.amazonaws.com/StagedProduct...,NaN,NaN,NaN,NaN,NaN,This is the Original Product Resolution (OPR) ...,processingUrl,2022-03-23
1,USGS Original Project Resolution LA_Amite_2018...,This is the Original Product Resolution (OPR) ...,623b83ead34e915b67d00a48,ScienceBase,NaN,gda,https://www.sciencebase.gov/catalog/item/623b8...,https://prd-tnm.s3.amazonaws.com/index.html?pr...,2019-11-07,2022-03-23T15:09:18.128-06:00,...,NaN,https://prd-tnm.s3.amazonaws.com/StagedProduct...,NaN,NaN,NaN,NaN,NaN,This is the Original Product Resolution (OPR) ...,processingUrl,2022-03-23
2,USGS Original Project Resolution LA_Amite_2018...,This is the Original Product Resolution (OPR) ...,623b860cd34e915b67d00e93,ScienceBase,NaN,gda,https://www.sciencebase.gov/catalog/item/623b8...,https://prd-tnm.s3.amazonaws.com/index.html?pr...,2019-11-07,2022-03-23T15:13:05.955-06:00,...,NaN,https://prd-tnm.s3.amazonaws.com/StagedProduct...,NaN,NaN,NaN,NaN,NaN,This is the Original Product Resolution (OPR) ...,processingUrl,2022-03-23
3,USGS Original Project Resolution LA_Amite_2018...,This is the Original Product Resolution (OPR) ...,623b8840d34e915b67d0146c,ScienceBase,NaN,gda,https://www.sciencebase.gov/catalog/item/623b8...,https://prd-tnm.s3.amazonaws.com/index.html?pr...,2019-11-07,2022-03-23T15:18:43.361-06:00,...,NaN,https://prd-tnm.s3.amazonaws.com/StagedProduct...,NaN,NaN,NaN,NaN,NaN,This is the Original Product Resolution (OPR) ...,processingUrl,2022-03-23
4,USGS Original Project Resolution LA_Amite_2018...,This is the Original Product Resolution (OPR) ...,623b849cd34e915b67d00c4d,ScienceBase,NaN,gda,https://www.sciencebase.gov/catalog/item/623b8...,https://prd-tnm.s3.amazonaws.com/index.html?pr...,2019-11-07,2022-03-23T15:11:09.152-06:00,...,NaN,https://prd-tnm.s3.amazonaws.com/StagedProduct...,NaN,NaN,NaN,NaN,NaN,This is the Original Product Resolution (OPR) ...,processingUrl,2022-03-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,USGS Original Project Resolution LA_Amite_2018...,This is the Original Product Resolution (OPR) ...,623b8a02d34e915b67d0192b,ScienceBase,NaN,gda,https://www.sciencebase.gov/catalog/item/623b8...,https://prd-tnm.s3.amazonaws.com/index.html?pr...,2019-11-07,2022-03-23T15:24:05.339-06:00,...,NaN,https://prd-tnm.s3.amazonaws.com/StagedProduct...,NaN,NaN,NaN,NaN,NaN,This is the Original Product Resolution (OPR) ...,processingUrl,2022-03-23
1131,USGS Original Project Resolution LA_Amite_2018...,This is the Original Product Resolution (OPR) ...,623b840bd34e915b67d00aae,ScienceBase,NaN,gda,https://www.sciencebase.gov/catalog/item/623b8...,https://prd-tnm.s3.amazonaws.com/index.html?pr...,2019-11-07,2022-03-23T15:09:36.103-06:00,...,NaN,https://prd-tnm.s3.amazonaws.com/StagedProduct...,NaN,NaN,NaN,NaN,NaN,This is the Original Product Resolution (OPR) ...,processingUrl,2022-03-23
1132,USGS Original Project Resolution LA_Amite_2018...,This is the Original Product Resolution (OPR) ...,623b89d3d34e915b67d01883,ScienceBase,NaN,gda,https://www.sciencebase.gov/catalog/item/623b8...,https://prd-tnm.s3.amazonaws.com/index.html?pr...,2019-11-07,2022-03-23T15:23:20.359-06:00,...,NaN,https://prd-tnm.s3.amazonaws.com/StagedProduct...,NaN,NaN,NaN,NaN,NaN,This is the Original Product Resolution (OPR) ...,processingUrl,2022-03-23
1133,USGS Original Project Resolution LA_Amite_2018...,This is the Original Product Resolution (OPR) ...,623b8515d34e915b67d00d3f,ScienceBase,NaN,gda,https://www.sciencebase.gov/catalog/item/623b8...,https://prd-tnm.s3.amazonaws.com/index.html?pr...,2019-11-07,2022-03-23T15:11:56.376-06:00,...,NaN,https://prd-tnm.s3.am

In [ ]:
# Instantiate WhiteboxTools
wbt = whitebox.WhiteboxTools()

# Set the working directory for input and output files
input_directory = project_dirs['Raw Data']
output_directory = project_dirs['Processed Data']

# Ensure output directory exists
os.makedirs(output_directory, exist_ok=True)

# List all .laz files in the input directory
laz_files = [f for f in os.listdir(input_directory) if f.endswith('.laz')]

# Process each .laz file
for laz_file in laz_files:
    input_file = os.path.join(input_directory, laz_file)
    intermediate_file = os.path.join(output_directory, laz_file.replace('.laz', '_dsm.tif'))
    dem_file = os.path.join(output_directory, laz_file.replace('.laz', '_dem.tif'))

    # 1. Convert LAZ to LAS (if needed)
    print(f"Converting {laz_file} to LAS format...")
    las_file = input_file.replace('.laz', '.las')
    wbt.lidar_translate(input_file, las_file, compress=False)
    
    # 2. Create a Digital Surface Model (DSM)
    print(f"Creating DSM from {las_file}...")
    wbt.lidar_point_density(las_file, output=f"{output_directory}/point_density.tif")
    wbt.lidar_rasterize(elevation=False, width=1.0, height=1.0, input=las_file, output=intermediate_file)
    
    # 3. Create a Digital Elevation Model (DEM) by filtering ground points
    print(f"Creating DEM from DSM...")
    wbt.lidar_ground_point_filter( input_file=las_file, output=dem_file, radius=10.0, slope_threshold=15.0 )
    
    print(f"Completed processing for {laz_file}")

print("DEM creation completed for all files.")
